# Pre-processing for Deliverable 3

## Pre-processing the new AQI data

In [66]:
import pandas as pd
import numpy as np

In [67]:
def calculate_daily_mean(minimum, maximum, q1, q3, median, count):
    n = count
    mean = (2.2 / (2.2 + n ** 0.75)) * ((minimum + maximum) / 2) + \
           (0.7 - (0.72 / n ** 0.55)) * ((q1 + q3) / 2) + \
           (0.3 + (0.72 / n ** 0.55) - (2.2 / (2.2 + n ** 0.75))) * median
    return mean

In [68]:
file_path = '../aqi_data/'

# List of all the files uploaded
file_names = ['02111.csv', '02118.csv', '02124.csv', '02127.csv', '02128.csv', '02130.csv', '02135.csv', '02139.csv']

In [69]:
# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['ZipCode', 'MeanAQI', 'AvgMaxAQI'])

In [71]:
for file_name in file_names:
    # Read the CSV file
    df = pd.read_csv(file_path + file_name)

    # Optional: Extract year from 'date' column if it exists
    # df['Year'] = pd.to_datetime(df['date']).dt.year

    # Extract zipcode from file name
    zipcode = file_name.split('.')[0]

    # Calculate the daily mean for each row
    df['Daily_Mean'] = df.apply(lambda x: calculate_daily_mean(x['min'], x['max'], x['q1'], x['q3'], x['median'], x['count']), axis=1)

    # Calculate the mean and average max AQI
    mean_aqi = df['Daily_Mean'].mean()
    avg_max_aqi = df['max'].mean()  # Modify as needed based on your data structure

    # Create a DataFrame for the current results and concatenate it with the main DataFrame
    current_results = pd.DataFrame({'ZipCode': [zipcode], 'MeanAQI': [mean_aqi], 'AvgMaxAQI': [avg_max_aqi]})
    results_df = pd.concat([results_df, current_results], ignore_index=True)

In [72]:
print(results_df.head())

  ZipCode   MeanAQI  AvgMaxAQI
0   02111  8.101017  37.325602
1   02118  9.796988  46.486448
2   02124  8.121950  22.361693
3   02127  9.149951  23.585692
4   02128  8.355616  22.265957


In [73]:
results_df.columns

Index(['ZipCode', 'MeanAQI', 'AvgMaxAQI'], dtype='object')

In [74]:
results_df['ZipCode'].unique()

array(['02111', '02118', '02124', '02127', '02128', '02130', '02135',
       '02139'], dtype=object)

In [75]:
results_df.to_csv('../2022AQIdata.csv', index=False)

## Pre-processing CDC health data

In [76]:
df = pd.read_csv('../cdc_data/Boston_CDCdata_Zipcode_2022.csv')

In [77]:
df.columns

Index(['ZCTA5', 'TotalPopulation', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI',
       'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev',
       'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI',
       'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev',
       'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI',
       'CERVICAL_CrudePrev', 'CERVICAL_Crude95CI', 'CHD_CrudePrev',
       'CHD_Crude95CI', 'CHECKUP_CrudePrev', 'CHECKUP_Crude95CI',
       'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI',
       'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev',
       'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI',
       'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev',
       'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI',
       'DEPRESSION_CrudePrev', 'DEPRESSION_Crude95CI', 'DIABETES_CrudePrev',
       'DIABETES_Crude95CI', 'GHLTH_CrudePrev', 'GHLTH_Crude95CI',
       'HIGHCHOL_CrudePrev', 'HIGHCHOL_Cru

In [78]:
# We are not interested in the confidence interval for each measure, so we will drop the columns that relate to that

df = df.loc[:, ~df.columns.str.endswith('CI')]

In [79]:
df.columns

Index(['ZCTA5', 'TotalPopulation', 'ACCESS2_CrudePrev', 'ARTHRITIS_CrudePrev',
       'BINGE_CrudePrev', 'BPHIGH_CrudePrev', 'BPMED_CrudePrev',
       'CANCER_CrudePrev', 'CASTHMA_CrudePrev', 'CERVICAL_CrudePrev',
       'CHD_CrudePrev', 'CHECKUP_CrudePrev', 'CHOLSCREEN_CrudePrev',
       'COLON_SCREEN_CrudePrev', 'COPD_CrudePrev', 'COREM_CrudePrev',
       'COREW_CrudePrev', 'CSMOKING_CrudePrev', 'DENTAL_CrudePrev',
       'DEPRESSION_CrudePrev', 'DIABETES_CrudePrev', 'GHLTH_CrudePrev',
       'HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev',
       'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev',
       'PHLTH_CrudePrev', 'SLEEP_CrudePrev', 'STROKE_CrudePrev',
       'TEETHLOST_CrudePrev', 'Geolocation'],
      dtype='object')

In [80]:
df.rename(columns={'ZCTA5': 'zip_code'}, inplace=True)

In [81]:
# Remove completely irrelevant columns

columns_to_remove = ['COREM_CrudePrev', 'COREW_CrudePrev']
df.drop(columns=columns_to_remove, inplace=True)



In [82]:
columns_to_remove = ['Geolocation']
df.drop(columns=columns_to_remove, inplace=True)

In [83]:
df.to_csv('../BostonFilteredCDCdata.csv', index=False)

## Merging the two datasets

In [84]:
aqi_data = pd.read_csv('../2022AQIdata.csv')
cdc_data = pd.read_csv('../BostonFilteredCDCdata.csv')

In [85]:
cdc_data = cdc_data.rename(columns={'zip_code': 'ZipCode'})

In [86]:
aqi_data['ZipCode'] = aqi_data['ZipCode'].astype(int)
cdc_data['ZipCode'] = cdc_data['ZipCode'].astype(int)

In [87]:
merged_data = pd.merge(aqi_data, cdc_data, on='ZipCode', how='outer')

In [88]:
# List of zip codes to be removed
zip_codes_to_remove = [2109, 2113, 2119, 2132, 2163, 2215]

# Remove rows with the specified zip codes
merged_data_filtered = merged_data[~merged_data['ZipCode'].isin(zip_codes_to_remove)]


In [89]:
merged_data.to_csv('../BostonCDCAQI.csv', index=False)